IMPORT DATA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import data from the generated dataset 
# generated in DataBuilding.ipnyb
data = pd.read_csv('test_data1.csv')
data.head()

,b1,b2,b3,b4,b5,class
0,[ 0 0 0 0 0 4 255 254],[153 0 0 0 0 0 0 154],[237 0 1 0 0 4 0 236],[181 255 0 0 0 0 255 180],[255 255 255 255 255 255 255 255],0
1,[ 0 0 0 0 0 4 255 254],[153 0 0 0 0 0 0 154],[237 0 4 0 0 4 0 236],[181 255 0 0 0 0 255 180],[255 255 255 255 255 255 255 255],1
2,[ 0 0 0 0 0 4 255 254],[153 0 0 0 0 0 0 154],[175 0 32 0 0 4 0 236],[181 255 0 0 0 0 255 180],[255 255 255 255 255 255 255 255],1
3,[ 0 0 0 0 0 4 255 254],[153 0 0 0 0 0 0 154],[175 0 128 0 0 4 0 236],[181 255 0 0 0 0 255 180],[255 255 255 255 255 255 255 255],0
4,[ 0 0 0 0 0 4 255 254],[153 0 0 0 0 0 0 154],[239 0 0 0 0 4 0 236],[181 254 1 0 0 0 255 180],[255 255 255 255 255 255 255 255],0


In [3]:
data.describe()

,class
count,60.000000
mean,0.516667
std,0.503939
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


Need to investigate origins and meaning of fen() encoding before examining further aspects of dataset

In [5]:
from sklearn import svm, decomposition

clf = svm.SVC()
y = data.pop('class')
clf.fit(data,y)

ValueError: could not convert string to float: '[255 255 255 255 255 255 255 255]'